In [1]:
# Testando API YouTube
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import csv 
from datetime import datetime, timezone

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.discovery import build

In [3]:
# Colocando nossa chave da API do YouTube
import os
from getpass import getpass

api_key = getpass("Insira aqui sua API key da OpenIA:")


In [18]:
# Função para buscar os vídeos em alta no YouTube e armazenar os dados em um DF
def videos_em_alta(n_videos: int, region: str):

    youtube = build('youtube', 'v3', developerKey=api_key)
    response = youtube.videos().list(
        part="snippet, statistics", 
        chart="mostPopular", 
        regionCode=region, 
        maxResults=n_videos,
            ).execute()
    videos = response.get("items", []) 

    #caso não há vídeos em alta exibirá a mensagem
    if not videos:
        print("Nenhum vídeo em alta foi encontrado! Ou informações não estão disponíveis")

    # dicionário para salvar as informações
    video_dict = {}

    for video in videos:
        

    # Extraindo as informações 
        video_id = video["id"]
        title = video["snippet"]["title"]
        views = video["statistics"]["viewCount"]
        likes = video["statistics"]["likeCount"]
        comments = video["statistics"]["commentCount"]
        published_at = video["snippet"]["publishedAt"]
        link_video = f"https://www.youtube.com/watch?v={video_id}"

        id = video["snippet"]['categoryId']
        response_cat = youtube.videoCategories().list(part="snippet",  id=id).execute()
        category = response_cat.get("items", [])[0]
        category_title = category["snippet"]["title"]

    # Armazenando as infos no dicionário
        video_dict[title] = {
            "views": views,
            "likes": likes,
            "comentarios": comments,
            "publicacao": published_at,
            "link_video": link_video,
            "categoria": category_title
        }
    # Salvando os dados em um DataFrame
    df = pd.DataFrame.from_dict(video_dict, orient="index")
    df = df.reset_index().rename(columns={'index': 'video'})

    # Convertendo a coluna 'publicacao' para datetime e criando uma nova coluna com o tempo postado
    current_time = datetime.now(timezone.utc)

    df['publicacao'] = pd.to_datetime(df['publicacao'])
    df["horas-publicacao"] = (current_time - df["publicacao"]).dt.total_seconds() / 3600
    df["horas-publicacao"] = df["horas-publicacao"].round()
    return df


In [5]:
# função para fazer update diário dos novos vídeos
def update_df(df, df_up):
    df = pd.concat([df,df_up])
    return df.reset_index(drop=True)


In [6]:
# Abrindo e salvando o df em uma variável
df = videos_em_alta(10, 'BR')
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao
0,Los Angeles Lakers x Miami Heat | AO VIVO COM ...,656498,64951,34,2023-11-07 03:09:15+00:00,https://www.youtube.com/watch?v=3uM5_hRwU78,Sports,13.0
1,MEU NOIVADO e CASAMENTO… DEUS NÃO UNE PESSOAS…...,318457,41702,11651,2023-11-06 20:43:48+00:00,https://www.youtube.com/watch?v=XhLqSF_KZA8,Entertainment,19.0
2,NOVEMBRO AZUL SINCERO,850865,116628,6451,2023-11-06 14:00:28+00:00,https://www.youtube.com/watch?v=iMmUh1BzxSs,Entertainment,26.0
3,"Character Demo - ""Furina: All the World's a St...",1387071,199818,9659,2023-11-07 04:00:04+00:00,https://www.youtube.com/watch?v=EN79SfbcvIE,Gaming,12.0
4,Pomni e Jax RESPONDEM no The Amazing Digital C...,183775,12398,1028,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=p0N_Xc9fQzo,Gaming,19.0
5,Jung Kook: Standing Next to You | The Tonight ...,1481473,288162,12769,2023-11-07 05:28:44+00:00,https://www.youtube.com/watch?v=6QzMbiPgYOg,Comedy,11.0
6,Jax e Pomni Respodem no The Amazing Digital Ci...,208763,10607,1187,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=_QnjC3xXqA0,Entertainment,19.0
7,The Marvels | Final Trailer | In Theaters Friday,1498886,56961,6328,2023-11-07 02:54:14+00:00,https://www.youtube.com/watch?v=uwmDH12MAA4,Entertainment,13.0
8,VASCO 1 X 0 BOTAFOGO | MELHORES MOMENTOS | 32ª...,1006368,31946,3924,2023-11-07 00:04:40+00:00,https://www.youtube.com/watch?v=zkR2h6Z3F-8,Sports,16.0
9,Ludmilla canta o Hino Nacional no GP de São Pa...,635468,7141,8766,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,Sports,47.0


In [23]:
df_us = videos_em_alta(10, 'US')


In [24]:

df = update_df(df, df_us)

In [25]:
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao
0,Los Angeles Lakers x Miami Heat | AO VIVO COM ...,656498,64951,34,2023-11-07 03:09:15+00:00,https://www.youtube.com/watch?v=3uM5_hRwU78,Sports,13.0
1,MEU NOIVADO e CASAMENTO… DEUS NÃO UNE PESSOAS…...,318457,41702,11651,2023-11-06 20:43:48+00:00,https://www.youtube.com/watch?v=XhLqSF_KZA8,Entertainment,19.0
2,NOVEMBRO AZUL SINCERO,850865,116628,6451,2023-11-06 14:00:28+00:00,https://www.youtube.com/watch?v=iMmUh1BzxSs,Entertainment,26.0
3,"Character Demo - ""Furina: All the World's a St...",1387071,199818,9659,2023-11-07 04:00:04+00:00,https://www.youtube.com/watch?v=EN79SfbcvIE,Gaming,12.0
4,Pomni e Jax RESPONDEM no The Amazing Digital C...,183775,12398,1028,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=p0N_Xc9fQzo,Gaming,19.0
5,Jung Kook: Standing Next to You | The Tonight ...,1481473,288162,12769,2023-11-07 05:28:44+00:00,https://www.youtube.com/watch?v=6QzMbiPgYOg,Comedy,11.0
6,Jax e Pomni Respodem no The Amazing Digital Ci...,208763,10607,1187,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=_QnjC3xXqA0,Entertainment,19.0
7,The Marvels | Final Trailer | In Theaters Friday,1498886,56961,6328,2023-11-07 02:54:14+00:00,https://www.youtube.com/watch?v=uwmDH12MAA4,Entertainment,13.0
8,VASCO 1 X 0 BOTAFOGO | MELHORES MOMENTOS | 32ª...,1006368,31946,3924,2023-11-07 00:04:40+00:00,https://www.youtube.com/watch?v=zkR2h6Z3F-8,Sports,16.0
9,Ludmilla canta o Hino Nacional no GP de São Pa...,635468,7141,8766,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,Sports,47.0


In [26]:
# converendo as colunas 'views', 'likes', 'comentarios' para int
df['views'] = df['views'].astype(int)
df['likes'] = df['likes'].astype(int)
df['comentarios'] = df['comentarios'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   video             20 non-null     object             
 1   views             20 non-null     int32              
 2   likes             20 non-null     int32              
 3   comentarios       20 non-null     int32              
 4   publicacao        20 non-null     datetime64[ns, UTC]
 5   link_video        20 non-null     object             
 6   categoria         20 non-null     object             
 7   horas-publicacao  20 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(1), int32(3), object(3)
memory usage: 1.1+ KB


In [27]:
# calculando a quantidade de views por hora
df["views/hour"] = (df["views"]/df["horas-publicacao"]).round(2)
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao,views/hour
0,Los Angeles Lakers x Miami Heat | AO VIVO COM ...,656498,64951,34,2023-11-07 03:09:15+00:00,https://www.youtube.com/watch?v=3uM5_hRwU78,Sports,13.0,50499.85
1,MEU NOIVADO e CASAMENTO… DEUS NÃO UNE PESSOAS…...,318457,41702,11651,2023-11-06 20:43:48+00:00,https://www.youtube.com/watch?v=XhLqSF_KZA8,Entertainment,19.0,16760.89
2,NOVEMBRO AZUL SINCERO,850865,116628,6451,2023-11-06 14:00:28+00:00,https://www.youtube.com/watch?v=iMmUh1BzxSs,Entertainment,26.0,32725.58
3,"Character Demo - ""Furina: All the World's a St...",1387071,199818,9659,2023-11-07 04:00:04+00:00,https://www.youtube.com/watch?v=EN79SfbcvIE,Gaming,12.0,115589.25
4,Pomni e Jax RESPONDEM no The Amazing Digital C...,183775,12398,1028,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=p0N_Xc9fQzo,Gaming,19.0,9672.37
5,Jung Kook: Standing Next to You | The Tonight ...,1481473,288162,12769,2023-11-07 05:28:44+00:00,https://www.youtube.com/watch?v=6QzMbiPgYOg,Comedy,11.0,134679.36
6,Jax e Pomni Respodem no The Amazing Digital Ci...,208763,10607,1187,2023-11-06 20:40:00+00:00,https://www.youtube.com/watch?v=_QnjC3xXqA0,Entertainment,19.0,10987.53
7,The Marvels | Final Trailer | In Theaters Friday,1498886,56961,6328,2023-11-07 02:54:14+00:00,https://www.youtube.com/watch?v=uwmDH12MAA4,Entertainment,13.0,115298.92
8,VASCO 1 X 0 BOTAFOGO | MELHORES MOMENTOS | 32ª...,1006368,31946,3924,2023-11-07 00:04:40+00:00,https://www.youtube.com/watch?v=zkR2h6Z3F-8,Sports,16.0,62898.00
9,Ludmilla canta o Hino Nacional no GP de São Pa...,635468,7141,8766,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,Sports,47.0,13520.60


In [28]:
# Data de hoje
date = df['publicacao'].dt.date[0]

In [31]:
# salvando o arquivo em um csv

df.to_csv(f'dados/top_10_youtube_{date}.csv', sep=';',index=False)